# Data Engineering

In [1]:
# import dependencies
import pandas as pd


In [2]:
# define data inputs
stations_data = "raw_data/hawaii_stations.csv"
measurements_data = "raw_data/hawaii_measurements.csv"

In [3]:
# open stations_data and measuremnts_data.  Replace all NaN, if any
stations_orig_pd = pd.read_csv(stations_data)
stations_orig_pd.fillna(0, inplace=True)

measurements_orig_pd = pd.read_csv(measurements_data)
measurements_orig_pd.fillna(0, inplace=True)

In [4]:
# write cleaned csv to and output file prefixed by clean
stations_orig_pd.to_csv("raw_data/clean_stations.csv", index=False)
measurements_orig_pd.to_csv("raw_data/clean_measurements.csv", index=False)

In [5]:
# using pandas read cleaned csv
stations_df = pd.read_csv("raw_data/clean_stations.csv")
measurements_df = pd.read_csv("raw_data/clean_measurements.csv")

# Database Engineering

In [6]:
# setup dependencies for sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from sqlalchemy import Table
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import relationship

from sqlalchemy import Column, Integer, String, Float, Numeric, Date

In [7]:
# setup station class
# station	name	latitude	longitude	elevation
class Stations(Base):
  __tablename__ = "stations"
  station = Column(String(100), primary_key=True)
  name = Column(String(300))
  latitude = Column(Float)
  longitude = Column(Float)
  elevation = Column(Float)
    
# setup masurements class
# station	date	prcp	tobs
class Measurements(Base):
  __tablename__ = "measurements"
  measurement_id = Column(Integer, primary_key=True, autoincrement=True)
  station = Column(String(100))
  date = Column(Date)
  prcp = Column(Float)
  tobs = Column(Integer)    


In [8]:
# Create Database Connection
# ----------------------------------
#  Establish SQL Lite connection
DB_CONN_URI_DEFAULT = 'sqlite:///hawaii.sqlite'

engine = create_engine(DB_CONN_URI_DEFAULT)




In [9]:
# Create a Table instance one for each weather station
metadata = MetaData(engine)
# Declare a table
for index, row in stations_df.iterrows():
    table = Table(row['name'].split( )[0],metadata,
#               Column('id',Integer, primary_key=True),
#               Column('name',String))
                  Column('id', Integer, primary_key=True, autoincrement=True),
                  Column('name', String(300)),
                  Column('latitude',Float),
                  Column('longitude',Float),
                  Column('elevation', Float),
                  Column('station', String(100)),
                  Column('date', Date),
                  Column('prcp', Float),
                  Column('tobs', Integer))



In [10]:
# create connection
conn = engine.connect()
# Create all tables
Base.metadata.create_all(conn)
metadata.create_all(conn)
# for _t in metadata.tables:
#    print ("Table: %s" % _t)

In [11]:
# populate stations table from csv file
stations_df.to_sql(con=conn, index=False, name='stations', if_exists='append', chunksize=20000)
# for s in stations_df:
#     s.to_sql(name='stations', if_exists='append',index=False, con=conn)

In [12]:
# populate measurements table from csv file
measurements_df.to_sql(con=conn, index=False, name='measurements', if_exists='append', chunksize=20000)
# for m in measurements_df:
#     m.to_sql(name='measurements', if_exists='append', con=conn)

In [13]:
weather_df = stations_df.merge(measurements_df, on='station', how='inner')
for index, row in stations_df.iterrows():
    tablename = row['name'].split( )[0].lower()
    weatherByName_df = weather_df[weather_df['name']==row['name']]
    weatherByName_df.to_sql(name=tablename, if_exists='append', index=False, con=conn, chunksize=20000)
    print(row['name'])


WAIKIKI 717.2, HI US
KANEOHE 838.1, HI US
KUALOA RANCH HEADQUARTERS 886.9, HI US
PEARL CITY, HI US
UPPER WAHIAWA 874.3, HI US
WAIMANALO EXPERIMENTAL FARM, HI US
WAIHEE 837.5, HI US
HONOLULU OBSERVATORY 702.2, HI US
MANOA LYON ARBO 785.2, HI US
